In [47]:
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix


In [4]:
iris_data = datasets.load_iris()
x = iris_data['data']
y = iris_data['target']

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=True)
print('x train shape:', x_train.shape)
print('y train shape:', y_train.shape)
print('x test shape:', x_test.shape)
print('y test shape:', y_test.shape)


x train shape: (127, 4)
y train shape: (127,)
x test shape: (23, 4)
y test shape: (23,)


In [23]:
def loss(y, p):
    p = np.clip(p, 1e-15, 1 - 1e-15)
    return - y * np.log(p) - (1 - y) * np.log(1 - p)

def loss_grad(y, p):
    p = np.clip(p, 1e-15, 1 - 1e-15)
    return - (y / p) + (1 - y) / (1 - p)


In [24]:
class GradientBoostingClassifier():

    def __init__(self, n, lr):
        self.trees = []
        self.n = n
        self.lr = lr
        for _ in range(n):
            tree = DecisionTreeRegressor(max_depth=1)
            self.trees.append(tree)

    def fit(self, X, y):
        y_pred = np.full(np.shape(y), np.mean(y, axis=0))
        for i in range(self.n):
            gradient = loss_grad(y, y_pred)
            self.trees[i].fit(X, gradient)
            update = self.trees[i].predict(X)
            y_pred -= self.lr * update

    def predict(self, X):
        y_pred = np.array([])
        for tree in self.trees:
            update = tree.predict(X)
            update = self.lr * update
            y_pred = -update if not y_pred.any() else y_pred - update
        y_pred = np.exp(y_pred) / np.expand_dims(np.sum(np.exp(y_pred), axis=1), axis=1)
        y_pred = np.argmax(y_pred, axis=1)
        return y_pred



In [49]:
gbc = GradientBoostingClassifier(4, lr=1)

one_hot_y = np.zeros((y_train.shape[0], len(np.unique(y_train))))
for i in range(one_hot_y.shape[0]):
    one_hot_y[i][y_train[i]] = 1

gbc.fit(x_train, one_hot_y)
pred = gbc.predict(x_test)

results = precision_recall_fscore_support(y_test, pred)
metrics = ['Precision', 'Recall', 'F']
for idx, metric in enumerate(metrics):
    print('{}:'.format(metric))
    print('\tClass 0: {}'.format(results[idx][0]))
    print('\tClass 1: {}'.format(results[idx][1]))
    print('\tClass 2: {}'.format(results[idx][2]))

cm = confusion_matrix(y_test, pred)
print('\nConfusion Matrix:')
print(cm)


Precision:
	Class 0: 1.0
	Class 1: 1.0
	Class 2: 0.8888888888888888
Recall:
	Class 0: 1.0
	Class 1: 0.8571428571428571
	Class 2: 1.0
F:
	Class 0: 1.0
	Class 1: 0.923076923076923
	Class 2: 0.9411764705882353

Confusion Matrix:
[[8 0 0]
 [0 6 1]
 [0 0 8]]
